### 1. Import Packages

In [2]:
# Librerie di base
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Librerie per la Hyperparameter Optimization
from sklearn.model_selection import GridSearchCV

# Librerie per il Machine Learning
from sklearn.naive_bayes import GaussianNB
import sklearn.metrics as metrics
from sklearn.model_selection import KFold, train_test_split, cross_val_score

# Librerie per la Features Selection
from sklearn.feature_selection import SelectKBest, f_classif

# Librerie per la PCA
from sklearn.decomposition import PCA

print('All packages successfully loaded')

All packages successfully loaded


### 2. Load Data & Peak Sheet

In [3]:
df = pd.read_csv('../../../data/ST001937.csv')

# Rimuoviamo le colonne che non ci servono
df.drop(columns=["Sample ID", "RAW_FILE_NAME"], inplace=True)

# Visualizziamo le prime 5 righe del dataset
df.head()

,Pheotypes,"1,3,5(10)-estratrien-3,6- beta-17-beta-triol","1,5-anhydroglucitol",17-alpha-20-alpha-dihydroxy-4-pregnen-3-one-1,17-alpha-20-alpha-dihydroxy-4-pregnen-3-one-2,17-alpha-20-alpha-dihydroxy-4-pregnen-3-one-3,17-alpha-20-alpha-dihydroxy-4-pregnen-3-one-4,17-alpha-20-alpha-dihydroxy-4-pregnen-3-one-5,17-alpha-20-alpha-dihydroxy-4-pregnen-3-one-6,1-hexadecanol,...,tyrosine-1,tyrosine-2,urea-1,urea-2,urea-3,urea-4,uridine,valine,xanthine,xanthosine
0,Benign SPNS,0.437408,0.663764,0.700609,0.096924,0.173891,0.799124,0.573520,1.105910,0.677833,...,0.689488,0.037373,0.595031,0.234936,0.589201,0.127788,0.113960,0.216687,0.770860,0.174060
1,Benign SPNS,5.064922,0.653161,0.883121,0.056551,0.932342,0.394648,0.772451,0.623514,0.769287,...,0.537845,0.016517,0.484474,0.564398,0.482784,0.119055,0.009865,0.164879,0.456740,0.320723
2,Benign SPNS,1.046057,0.798040,0.838630,0.283674,0.119269,0.032103,0.807268,0.339667,1.104029,...,0.679699,1.079720,0.901623,0.559234,0.866856,0.159342,0.026922,0.252849,0.473077,0.158255
3,Benign SPNS,2.530700,1.318840,1.074717,0.431186,1.773602,0.627714,2.042268,0.913226,0.741269,...,0.837193,0.522064,1.433745,1.442062,1.406062,0.231101,0.278952,0.301997,0.940990,1.095022
4,Benign SPNS,0.027033,1.031146,0.863596,0.201422,0.222126,0.928862,1.004769,0.403102,0.790744,...,0.826921,0.462153,1.040359,1.154850,1.034635,0.198105,0.868354,0.302138,1.208439,0.557425


### 2.1 Data Cleaning

Notiamo che il dataset non presenta alcun nullo, di conseguenza non dobbiamo effettuare nessuna operazione di imputazione. Dobbiamo soltanto modificare i valori testuali della feature Pheotypes in valori numerici.

In [4]:
# Visualizziamo le colonne con dati mancanti
print(df.isnull().sum())
print("----------------------------------------------------")
print(df.isnull().sum().sum())

df['Pheotypes'] = df['Pheotypes'].apply(lambda x: 0 if x == 'Healthy Controls' else 1 if x == 'Benign SPNS' else 2)

Pheotypes                                        0
1,3,5(10)-estratrien-3,6- beta-17-beta-triol     0
1,5-anhydroglucitol                              0
17-alpha-20-alpha-dihydroxy-4-pregnen-3-one-1    0
17-alpha-20-alpha-dihydroxy-4-pregnen-3-one-2    0
                                                ..
urea-4                                           0
uridine                                          0
valine                                           0
xanthine                                         0
xanthosine                                       0
Length: 546, dtype: int64
----------------------------------------------------
0


### 3. Extract X & Y

In [5]:
X = df.drop(columns=['Pheotypes'])
X_features_names = X.columns
y = df.Pheotypes

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 4. Initial Model Build

In [6]:
# Definiamo il modello GaussianNB con gli iperparametri di default
model = GaussianNB()

# Addestriamo il modello
model.fit(X_train, y_train)

# Eseguiamo le previsioni sui dati di test
y_pred = model.predict(X_test)

### 5. Initial Model Evalutation

In [7]:
# Valutiamo l'accuratezza del modello con gli iperparametri di default
accuracy = metrics.accuracy_score(y_test, y_pred)

# Visualizziamo le metriche
print(f'Accuratezza: {accuracy}')

Accuratezza: 0.5301724137931034


Le prestazioni iniziali sono ottime: abbiamo un'accuratezza del 86%. Proviamo con la K-Fold Cross Validation.

In [8]:
# Testiamo diverse configurazioni di K
max = 0
k_best = 0
for k in range(5, 11): 
    kfolds = KFold(n_splits=k, shuffle=True, random_state=42)
    model = GaussianNB()
    scores = cross_val_score(model, X, y, cv=kfolds)
    mean = np.mean(scores)
    if mean > max: 
        max = mean
        k_best = k
    print(f"K={k}, Accuratezza Media: {mean}")
    
print("-------------------------------------------")
print(f"K ottimale: {k_best}")
print(f"Accuratezza media massima: {max}")

K=5, Accuratezza Media: 0.5784482758620689


K=6, Accuratezza Media: 0.5784769332122571
K=7, Accuratezza Media: 0.5768059249986962
K=8, Accuratezza Media: 0.5741379310344827
K=9, Accuratezza Media: 0.5733473298880276
K=10, Accuratezza Media: 0.5767241379310345
-------------------------------------------
K ottimale: 6
Accuratezza media massima: 0.5784769332122571


Abbiamo ottenuto le prestazioni migliori con un numero di fold pari a 6.

In [9]:
# Creiamo l'oggetto K-Fold per la Cross-Validation con il numero di fold ottimale
kfolds = KFold(n_splits=k_best, shuffle=True, random_state=42)

### 6. Hyperparameters Optimization

L'**ottimizzazione degli iperparametri** è un passo fondamentale nello sviluppo di modelli predittivi robusti. Infatti, aderire ai parametri predefiniti impedisce ai modelli di raggiungere il massimo delle prestazioni. A tale scopo, utilizziamo la tecnica **Grid Search**.

### 7. Feature Selection

Per la feature selection, al contrario di altri dataset già esaminati, non possiamo utilizzare il test del chi-quadro in quanto esso supporta solo dati non negativi. Per questo motivo, proviamo a utilizzare la tecnica *F-Classif* calcola il valore *F-statistic* e il *p-value* associato per ogni feature rispetto alla variabile target. L'*F-statistic* misura la differenza tra le medie dei gruppi rispetto alla varianza dei dati all'interno dei gruppi. Il *p-value* è quindi utilizzato per valutare la significatività statistica della differenza osservata.

In [10]:
max_fs = 0 
k_fs = 0
best_features = []

for k in range(2, len(X.columns)):
    top_features = SelectKBest(score_func=f_classif, k=k).fit(X, y).get_support(indices=True)
    X_top = X.iloc[:, top_features]
    
    scores = cross_val_score(model, X_top, y, cv=kfolds, scoring='accuracy')
    mean = np.mean(scores)
    
    if mean > max_fs: 
        max_fs = mean
        k_fs = k
        best_features = X.columns[top_features].tolist()

    print(f"K={k}, Accuratezza Media: {mean}")

print(f"K ottimale: {k_fs}")
print(f"Accuratezza media ottimale: {max_fs}")
print(f"Features ottimali: {best_features}")

K=2, Accuratezza Media: 0.6611692751455585
K=3, Accuratezza Media: 0.7146252871107311
K=4, Accuratezza Media: 0.597408436159037
K=5, Accuratezza Media: 0.6620506383206026
K=6, Accuratezza Media: 0.6327519897441376
K=7, Accuratezza Media: 0.653437316382672
K=8, Accuratezza Media: 0.6258390755479586
K=9, Accuratezza Media: 0.6077399711553869
K=10, Accuratezza Media: 0.6172302049391948
K=11, Accuratezza Media: 0.6120533091181027
K=12, Accuratezza Media: 0.6249666150312483
K=13, Accuratezza Media: 0.6111852999305593
K=14, Accuratezza Media: 0.6215123835977423
K=15, Accuratezza Media: 0.6249621637020814
K=16, Accuratezza Media: 0.6370208144151844
K=17, Accuratezza Media: 0.6499697309616651
K=18, Accuratezza Media: 0.6491061731032887
K=19, Accuratezza Media: 0.6508510941367093
K=20, Accuratezza Media: 0.6508466428075423
K=21, Accuratezza Media: 0.6474057653615369
K=22, Accuratezza Media: 0.6525693071951285
K=23, Accuratezza Media: 0.6318795292274273
K=24, Accuratezza Media: 0.635324858002599

L'accuratezza più alta si ha con le migliori 541 features, solo 3 in meno rispetto a quelle originali. Il miglioramento di accuratezza è alquanto trascurabile.

In [11]:
X_top = X[best_features]
scores = cross_val_score(model, X_top, y, cv=kfolds, scoring='accuracy')
print(f'Accuratezza media: {scores.mean()}')

Accuratezza media: 0.7146252871107311


### 8. PCA

In [12]:
pca = PCA()
pca.fit(X)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.99) + 1
print("Numero di componenti principali:", d)

pca = PCA(n_components=d)
X_reduced = pca.fit_transform(X)

Numero di componenti principali: 86


In [13]:
scores = cross_val_score(model, X_reduced, y, cv=kfolds, scoring='accuracy')
print(f'Accuratezza media: {scores.mean()}')

Accuratezza media: 0.36125652119722956


La PCA non ci aiuta nelle nostre analisi. Infatti, è impossibile arrivare a plottare le componenti principali in quanto saranno sempre maggiori di 3.

### 9. Results

Visualizziamo i risultati migliori.